# Алгоритмы интеллектуальной обработки больших объемов данных
## Семестровый проект


In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from sklearn.preprocessing import scale

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (14,7)

## О hearthstone

[Hearthstone](http://eu.battle.net/hearthstone/ru/) - карточная он-лайн игра по мотивам вселенной Warcraft.

Каждый игрок играет за персонажа определенного класса и собирает колоду карт которую может разыгрывать во время игры. Для дальнейших деталей обратитесь к [wiki](https://ru.wikipedia.org/wiki/Hearthstone), посмотрите youtube или поиграйте сами (но не долго =) ).

Теми или иными способами игрок может отнимать жизни у своего оппонента, таким образом цель раунда - побить другого игрока.

<center><img src='http://ps3hits.ru/wp-content/uploads/2015/08/hearthstone-game-sshot-1.jpg'></center>

## Постановка задачи

В рамках конференции [AAIA 17](https://fedcsis.org/2017/aaia) было запущено [соревнование](https://knowledgepit.fedcsis.org/contest/view.php?id=120) по предсказанию исхода раунда в heartstone. 

Используя признаки, которые описывают текущее состояние раунда, необходимо предсказать **вероятность** победы игрока в этом раунде.

Качество модели измеряется с помощью **ROC-AUC**

### Правила
* Объединяться в команды по правилам конкурса можно, но по правилам нашего курса - нельзя)
* Вы можете использовать любой алгоритм, даже неизученный в нашем курсе (если на защите сможете объяснить, как он работает)
* Имейте ввиду, что вы должны отправить результаты строго до 14 мая 23:59 GMT на сайт соревнования (раздел submission). Без результата в leaderboard оценка выставлена не будет.

## Данные

Для того, чтобы получить данные, необходимо зарегистрироваться на сайте соревнования. 

Данные содержат набор информации о раунде в некоторый момент времени: жизни игрока и оппонента, номер хода, карты на столе, карты в руке игрока, количество карт в руке оппонента и тп.<br/>
По игроку №1 (далее просто **игрок**) помимо всего прочего известен набор карт "в руке".</br>
По игроку №2 (далее просто **оппонент**) эта информация отсутствует.</br>


Данные поставляются в двух форматах:
* В формате json содержится полная информация по текущему состоянию раунда
* В табличном формате упрощенная аггрерированная информация по текущему состоянию раунда

В данных для обучению содержится 2 млн. игр, разбитых на 4 файла. Названия признаков говорят сами за себя.
Целевой признак - `decision` (1 - победил игрок, 0 - победил оппонент).

In [2]:
# Загрузка данных целиком
df_list = []
for chunk in range(1,3):
    filepath = '/media/ivan/Acer/Ivan/sphere/Hearthstone/trainingData_tabular_chunk%d.csv' % chunk
    df_list.append(pd.read_csv(filepath, sep=','))

df_data = pd.concat(df_list)

In [3]:
df_data.shape

(1000000, 45)

In [4]:
# для удобства
df_data.columns = df_data.columns.str.replace('.', '_')
df_data = df_data.set_index('gamestate_id')

In [5]:
df_data.head().T

gamestate_id,4687346.0,3270826.0,3189487.0,4098946.0,2661127.0
decision,1,1,1,1,0
turn,14,7,5,8,13
opponent_armor,0,0,3,0,0
opponent_attack,0,0,0,0,0
opponent_hero_card_id,798,754,612,390,25
opponent_hp,5,22,29,26,30
opponent_special_skill_used,0,0,0,0,0
opponent_weapon_durability,0,0,0,0,0
opponent_crystals_all,10,6,4,7,10
opponent_crystals_current,10,6,1,7,1


In [6]:
# Распределение классов
y = df_data.decision.values
df_data.drop(df_data[['decision']], axis=1, inplace=True)

y.mean()
# Примерно поровну

0.50461

<h2>Подготовка данных для линейных методов</h2>

hero_card_id - категориальная переменная

In [7]:
un_opp = df_data['opponent_hero_card_id'].unique()
print u'Уникальные карты оппонента:', un_opp
print u'Их число:', len(un_opp)
un_pl = df_data['player_hero_card_id'].unique()
print u'Уникальные карты игрока:', un_pl
print u'Их число:', len(un_pl)

Уникальные карты оппонента: [ 798  754  612  390   25 1235  494  981  326]
Их число: 9
Уникальные карты игрока: [ 981  754  612  494   25 1235  326  390  798]
Их число: 9


Векторизуем категориальные признаки - opponent_hero_card_id и player_hero_card_id. Эти поля могут принимать по 9 значений. Создадим для каждого из значений отдельную колонку, в которой для каждой игры будет стоять либо 0, либо 1, в зависимости от того, эта ли карта используется в данной игре:

In [8]:
from sklearn.preprocessing import OneHotEncoder

opponent = OneHotEncoder().fit_transform(df_data['opponent_hero_card_id'].values.reshape(-1, 1)).toarray().astype(int)
player = OneHotEncoder().fit_transform(df_data['player_hero_card_id'].values.reshape(-1, 1)).toarray().astype(int)

for i in xrange(len(opponent[0])):
    df_data['opponent_hero_card_{}'.format(i)] = opponent[:,i]
    df_data['player_hero_card_{}'.format(i)] = player[:,i]

Удалим ставшие ненужными колонки opponent_hero_card_id и player_hero_card_id:

In [9]:
df_data.drop(df_data[['opponent_hero_card_id', 'player_hero_card_id']], axis=1, inplace=True)

Теперь наши данные выглядят так:

In [10]:
df_data.head().T

gamestate_id,4687346.0,3270826.0,3189487.0,4098946.0,2661127.0
turn,14,7,5,8,13
opponent_armor,0,0,3,0,0
opponent_attack,0,0,0,0,0
opponent_hp,5,22,29,26,30
opponent_special_skill_used,0,0,0,0,0
opponent_weapon_durability,0,0,0,0,0
opponent_crystals_all,10,6,4,7,10
opponent_crystals_current,10,6,1,7,1
opponent_deck_count,11,19,22,17,12
opponent_fatigue_damage,0,0,0,0,0


Преобразуем dataframe в матрицу признаков:

In [11]:
X = df_data.values

print X.shape, y.shape

(1000000, 59) (1000000,)


Нормировка

In [14]:
X = scale(X, copy=False)

/home/ivan/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


<h2>Логистическая регрессия</h2>

Найдем оптимальные параметры модели на части данных. Будем считать, что данные распределены равномерно и для всей выборки параметры будут такими же. 

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [16]:
estimator = LogisticRegression(random_state=None)
params = {'C': np.logspace(-3, 3, 20)}
grid = GridSearchCV(estimator, params).fit(X[:100000], y[:100000])

In [18]:
grid.best_estimator_

LogisticRegression(C=0.0020691380811147901, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [ ]:
plt.plot()

Обучим классификатор с наилучшими параметрами на полной обучающей выборке:

In [16]:
model = LogisticRegression(C=1.0).fit(X[:1000000], y[:1000000])

Точность 0.7896

<h2>Random Forest</h2>

In [5]:
from sklearn.ensemble import RandomForestClassifier

In [7]:
%%time
model = RandomForestClassifier(n_estimators=100, max_features=10).fit(X, y)

CPU times: user 2min 23s, sys: 228 ms, total: 2min 23s
Wall time: 2min 23s


Лучшая точность 0.7569

<h2>Понижение размерности</h2>

In [13]:
from sklearn.decomposition import PCA

In [15]:
pca = PCA(n_components=20, copy=False).fit(X)

In [17]:
X = pca.transform(X)

In [18]:
X.shape

(1000000, 20)

In [21]:
estimator = LogisticRegression(random_state=None)
params = {'C': np.logspace(-3, 3, 20)}
grid = GridSearchCV(estimator, params).fit(X, y)

In [22]:
grid.best_estimator_

LogisticRegression(C=483.29302385717523, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [23]:
grid.best_score_

0.70063500000000001

In [26]:
model = LogisticRegression(C=1.0).fit(X, y)

In [33]:
X_test = pca.transform(X_test)

Точность 0.7865

<h2>SVM</h2>

In [ ]:
from sklearn.svm import SVC

In [ ]:
model = SVC(kernel='linear').fit(X, y)

<h2>Feature selection</h2>

In [ ]:
sns.heatmap(df_data.corr())

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
%%time
model = RandomForestClassifier(n_estimators=20, max_features=20).fit(X, y)

In [ ]:
features = {}
columns = list(df_data.columns)
for i in xrange(len(columns)):
    features[columns[i]] = cls.feature_importances_[i]
    
import operator
sorted_features = sorted(features.items(), key=operator.itemgetter(1), reverse=True)
sorted_features

<h2>Feature engineering</h2>

In [ ]:
df_data['hp_ratio'] = df_data['hp_ratio']/df_data['hp_ratio']
X = df_data.values
X.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
%%time
model = RandomForestClassifier(n_estimators=20, max_features=20).fit(X, y)

In [ ]:
features = {}
columns = list(df_data.columns)
for i in xrange(len(columns)):
    features[columns[i]] = cls.feature_importances_[i]
    
import operator
sorted_features = sorted(features.items(), key=operator.itemgetter(1), reverse=True)
sorted_features

<h2>Тестовые данные</h2>

In [ ]:
filepath = '/media/ivan/Acer/Ivan/sphere/Hearthstone/testData_tabular.csv'
df_test = pd.read_csv(filepath, sep=',')

df_test.columns = df_test.columns.str.replace('.', '_')
df_test = df_test.set_index('gamestate_id')

df_test.drop(df_test[['decision']], axis=1, inplace=True)

In [ ]:
opponent = OneHotEncoder().fit_transform(df_test['opponent_hero_card_id'].values.reshape(-1, 1)).toarray().astype(int)
player = OneHotEncoder().fit_transform(df_test['player_hero_card_id'].values.reshape(-1, 1)).toarray().astype(int)

for i in xrange(len(opponent[0])):
    df_test['opponent_hero_card_{}'.format(i)] = opponent[:,i]
    df_test['player_hero_card_{}'.format(i)] = player[:,i]
    
df_test.drop(df_test[['opponent_hero_card_id', 'player_hero_card_id']], axis=1, inplace=True)

In [ ]:
X_test = df_test.values
X_test.shape

In [ ]:
X_test = scale(X_test, copy=False)

<h2>Применение модели</h2>

In [ ]:
scores = model.predict_proba(X_test)
scores.shape

<h2>Комбинация моделей</h2>

In [ ]:
filepath = '/media/ivan/Acer/Ivan/sphere/Hearthstone/'
df1 = pd.read_csv(filepath+'hearthstone_tree1.txt', header=False)
df1 = pd.read_csv(filepath+'hearthstone_tree1.txt', header=False)

df1 = (df1 + df2)/2
scores = df1.values
scores.shape

<h2>Запись результатов в файл:</h2>

In [11]:
f = open('/media/ivan/Acer/Ivan/sphere/Hearthstone/hearthstone.txt', 'w')
s = ''
for i in xrange(len(scores)):
    s += str(scores[i, 1]) + '\n'
f.write(s)
f.close()

Результаты на странице конкурса: <br>
i.kozlov<br>
0.7896